In [ ]:
#Dependencies
library(readxl)
library(GenomicRanges)
library(rtracklayer)

In [2]:
### loading significant tDMRs from each scenario
resq1 <- read.table("resq1.txt", header = TRUE, sep = "\t")   
resq2<- read.table("resq2.txt", header = TRUE, sep = "\t")
resq3 <- read.table("resq3.txt", header = TRUE, sep = "\t")
resq4<- read.table("resq4.txt", header = TRUE, sep = "\t")
resq5 <- read.table("resq5.txt", header = TRUE, sep = "\t")
resq6<- read.table("resq6.txt", header = TRUE, sep = "\t")
resq7 <- read.table("resq7.txt", header = TRUE, sep = "\t")
resq8<- read.table("resq8.txt", header = TRUE, sep = "\t")
resq75 <- read.table("resq7_0.05.txt", header = TRUE, sep = "\t")
resq85<- read.table("resq8_0.05.txt", header = TRUE, sep = "\t")

test<-resq75

[1] 242   8

,DMRnumber,chr,st,end,intercept,f,pval,qval
,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,7478,chr20,60751975,60752320,2.1519394,38.33832,2.804548e-07,0.001854872
2,12381,chr17,5882814,5883013,2.0168519,36.42646,4.623939e-07,0.001854872
3,11235,chr1,25472270,25472754,0.3109544,33.73726,9.559373e-07,0.002034627
4,4140,chr9,139414758,139415013,0.8740158,32.83552,1.227302e-06,0.002034627
5,12168,chr17,37887321,37887546,0.5769506,32.71863,1.268011e-06,0.002034627
6,7490,chr20,32716846,32717096,3.2589058,28.36667,4.456243e-06,0.005958671


In [3]:
# Combining data frames with an added column for origin
Alldf <- do.call(rbind, lapply(seq_along(list(resq1, resq2, resq3, resq4, resq5 ,resq6, resq7, resq8)), function(i) {
  df <- list(resq1, resq2, resq3, resq4,resq5, resq6, resq7, resq8)[[i]]
  df$Origin <- paste0("resq", i)
  df}))
sorted_df <- Alldf[order(Alldf$qval), ]

# Checking for repetitive DMRs (common between scenarios)
repetitive_dmrs <- Alldf[duplicated(Alldf), ]
if (nrow(repetitive_dmrs) > 0) {
  cat("Repetitive DMRs found:\n")
  print(repetitive_dmrs)
} else { cat("No repetitive DMRs found.\n")}

#most significant markers from all scenarios
dim(Alldf)
filtered_df <- Alldf[Alldf$qval < 0.3, ]
sorted_df <- filtered_df[order(filtered_df$qval), ]
dim(sorted_df)
sorted_df[1:10,]

No repetitive DMRs found.


[1] 2554    9

[1] 2554    9

,DMRnumber,chr,st,end,intercept,f,pval,qval,Origin
,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
122,7478,chr20,60751975,60752320,2.1519394,38.33832,2.804548e-07,0.001854872,resq7
123,12381,chr17,5882814,5883013,2.0168519,36.42646,4.623939e-07,0.001854872,resq7
124,11235,chr1,25472270,25472754,0.3109544,33.73726,9.559373e-07,0.002034627,resq7
125,4140,chr9,139414758,139415013,0.8740158,32.83552,1.227302e-06,0.002034627,resq7
126,12168,chr17,37887321,37887546,0.5769506,32.71863,1.268011e-06,0.002034627,resq7
127,7490,chr20,32716846,32717096,3.2589058,28.36667,4.456243e-06,0.005958671,resq7
110,12707,chr16,10603065,10603332,6.6582115,43.26281,5.645787e-07,0.006285111,resq6
128,514,chr22,49552622,49552877,-1.8083804,27.12006,6.490423e-06,0.006593549,resq7
129,12528,chr5,150044642,150044875,1.1197275,26.57706,7.663729e-06,0.006593549,resq7


In [7]:

top1000 <- read_excel("C:/Users/Hiam/Desktop/DNA methylation atlas of normal human cell types/S4C_TOP1000.xlsx", sheet = 1) 
mine<-sorted_df
mine$st<-mine$st - 50
mine$end<-mine$end + 50
gr1 <- GRanges(seqnames = mine$chr,
               ranges = IRanges(start = mine$st, end = mine$end),
               beta = mine$qval)
atlas<-top1000
gr2 <- GRanges(seqnames = atlas$chr,
               ranges = IRanges(start = atlas$start, end = atlas$end),
               beta = atlas$Diff) 
overlaps <- findOverlaps(gr1, gr2)
common_regions <- pintersect(gr1[queryHits(overlaps)], gr2[subjectHits(overlaps)])
overlap_indices <- subjectHits(overlaps)
overlapping_rows <- top1000[overlap_indices, ]
dim(data.frame(overlapping_rows))
head(data.frame(overlapping_rows))

New names:
• `` -> `...15`


[1] 695  16

,Type,chr,start,end,startCpG,endCpG,position,Number.of.CpGs,Length,Target.meth.,Background.meth.,Diff,Genomic.class,Gene,...15,p.value
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<dbl>
1,Ovary-Ep,chr1,25473034,25473185,442230,442239,chr1:25473034-25473185,9CpGs,151bp,0.0353,0.663,0.6277,Intergenic,SYF2,NA,NA
2,Liver-Hep,chr20,61520337,61520582,25750679,25750693,chr20:61520337-61520582,14CpGs,245bp,0.0556,0.906,0.8504,exon,DIDO1,NA,NA
3,Liver-Hep,chr3,186333563,186333911,5941634,5941646,chr3:186333563-186333911,12CpGs,348bp,0.1110,0.911,0.8000,intron,AHSG,NA,NA
4,Pancreas-Beta,chr4,84021657,84022070,6769954,6769959,chr4:84021657-84022070,5CpGs,413bp,0.0138,0.851,0.8372,intron,PLAC8,NA,NA
5,Kidney-Ep,chr16,25698111,25698424,21490735,21490742,chr16:25698111-25698424,7CpGs,313bp,0.2620,0.814,0.5520,Intergenic,HS3ST4,NA,NA
6,Liver-Hep,chr21,46892633,46892991,26147499,26147514,chr21:46892633-46892991,15CpGs,358bp,0.1180,0.907,0.7890,intron,COL18A1,NA,NA


In [118]:
# all DMR inforamtion in 1 table (covid-19 analysis and atlas DMRs)
overlaps <- findOverlaps(gr1, gr2)
common_regions <- pintersect(gr1[queryHits(overlaps)], gr2[subjectHits(overlaps)])
overlap_indices <- subjectHits(overlaps)
overlapping_rows <- top1000[overlap_indices, ]
dmr_numbers <- mine$DMRnumber[queryHits(overlaps)]
overlapping_rows$DMRnumber <- dmr_numbers
for (col_name in c("chr", "st", "end", "intercept", "f", "pval", "qval")) {
  overlapping_rows[[col_name]] <- mine[[col_name]][match(overlapping_rows$DMRnumber, mine$DMRnumber)]}
dim(overlapping_rows)

#best DMRs as biomarkers
x<-subset(overlapping_rows, qval < 0.05)[,-c(7,15,16)]
dim(x)
x1<-subset(x, grepl("Pancreas|Liver|Blood|Colon|Breast|Gastric|Kidney", Type))
dim(x1)
x2<- x1[x1[, 12] != "Intergenic", ]
dim(x2)

[1] 695  22

[1] 50 19

[1] 36 19

[1] 23 19

In [125]:
#beta value difference calculation betweeen mild and severe
NAdelbed7= read.table("NAdelbed7.txt",sep=" ",header = T)
gname1 <-'abnormal'   # in sen4&5 : Ascending  in sen6: discharged in sen7&8 : abnormal
gname2 <-'normal'   # in sen4&5 : Descending in sen6: deceased  in sen7&8: normal
# beta difference calculation
NAdelbed7 <- read.table("NAdelbed7.txt", sep=" ", header = TRUE)
# Defining column names
abnormal_columns <- grep("^abnormal", colnames(NAdelbed7))
normal_columns <- grep("^normal", colnames(NAdelbed7))
NAdelbed7$mean_severe <- rowMeans(NAdelbed7[, abnormal_columns], na.rm = TRUE)
NAdelbed7$mean_mild <- rowMeans(NAdelbed7[, normal_columns], na.rm = TRUE)
# beta difference between SEVERE and MILD 
NAdelbed7$beta_diff <- NAdelbed7$mean_severe - NAdelbed7$mean_mild
s_table <- NAdelbed7[, c("DMRnumber", "mean_severe", "mean_mild", "beta_diff")]
s_sorted <- summary_table[order(summary_table$beta_diff, decreasing = TRUE), ]

# Merging x2 with s_sorted based on `DMRnumber`
merged_x2 <- merge(x2, s_sorted, by = "DMRnumber", all.x = TRUE)

# Replacing columns
merged_x2$f <- merged_x2$mean_severe
merged_x2$intercept <- merged_x2$mean_mild
merged_x2$st <- merged_x2$beta_diff
final_x2 <- merged_x2[, !(colnames(merged_x2) %in% c("mean_severe", "mean_mild", "beta_diff"))]
colnames(final_x2)[colnames(final_x2) == "f"] <- "mean_severe"
colnames(final_x2)[colnames(final_x2) == "intercept"] <- "mean_mild"
colnames(final_x2)[colnames(final_x2) == "st"] <- "beta_diff"

subset(final_x2, beta_diff < -25)
# saving "beta_difference_summary"

,DMRnumber,Type,chr,start,end,startCpG,endCpG,Number of CpGs,Length,Target meth.,Background meth.,Diff,Genomic class,Gene,beta_diff,mean_mild,mean_severe,pval,qval
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,3183,Gastric-Ep:Small-Int-Ep,chr10,104574094,104574452,15622048,15622053,5CpGs,73bp,0.0067,0.762,0.7553,exon,WBP1L,-36.35537,79.01216,42.65679,5.456546e-04,0.034859393
7,6254,Liver-Hep,chr12,12409218,12409700,17428008,17428017,9CpGs,544bp,0.1350,0.917,0.7820,intron,LRP6,-26.98481,83.86566,56.88085,8.276977e-04,0.040154873
11,7821,Liver-Hep,chr20,61520337,61520880,25750679,25750693,14CpGs,245bp,0.0556,0.906,0.8504,exon,DIDO1,-27.95108,92.71960,64.76852,1.507968e-05,0.007496507
12,8142,Colon-Ep:Gastric-Ep:Small-Int-Ep,chr16,70186562,70187168,21874363,21874374,11CpGs,308bp,0.0196,0.881,0.8614,intron,PDPR,-28.43064,92.30254,63.87190,1.395064e-03,0.048243401
13,8142,Gastric-Ep:Small-Int-Ep,chr16,70186562,70187168,21874363,21874374,11CpGs,308bp,0.0230,0.844,0.8210,intron,PDPR,-28.43064,92.30254,63.87190,1.395064e-03,0.048243401


In [ ]:
intergenic   235
intron  362
exon  61
promoter-TSS  37